# Fashion MNIST Image Classification - Azure ML SDK Training

In this Fashion MNIST notebook we introduce how to instrument your training process with the Azure ML SDK. 

This code will show how Azure ML SDK can support your machine learning project with:
* A central repository for your machine learning project
* Creating a cloud computer target and running your training in the cloud
* Whilst running your training in the cloud, add logging to the code to see in real time in your notebook the outputs and progress of the training on the remote compute in the cloud
* Saving your large datasets to azure storage so your training models can mount the data to the assigned training compute and have a 'one-source-of-truth' dataset for all your data science team to be using
* Finally registering this model with versioning so others can leverage it easily

This notebook is based off the great sample in the docs here: [https://docs.microsoft.com/en-us/azure/machine-learning/service/tutorial-train-models-with-aml](https://docs.microsoft.com/en-us/azure/machine-learning/service/tutorial-train-models-with-aml?WT.mc_id=aisummit-github-amynic)

## Import packages

In [17]:
%matplotlib inline
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

import azureml
from azureml.core import Workspace, Run

# check core SDK version number
print("Azure ML SDK Version: ", azureml.core.VERSION)

Azure ML SDK Version:  0.1.65


Load your Azure ML workspace from the config file you setup. The config file will look like below:

![Config File](./images/configfile.JPG "Config file")

## Load the workspace created

In [18]:
# load workspace configuration from the config.json file in the current folder.
ws = Workspace.from_config()
print(ws.name, ws.location, ws.resource_group, ws.location, sep = '\t')

Found the config file in: C:\Users\amynic\Documents\CloudDeveloperAdvocate\Deeplearning\ai-world-summit\azureml-sdk-fashion\config.json
amyboyd_ws	westeurope	deeplearning	westeurope


In [19]:
experiment_name = 'deeplearning_fashion'

from azureml.core import Experiment
exp = Experiment(workspace=ws, name=experiment_name)
print(exp)

Experiment(Name: deeplearning_fashion,
Workspace: amyboyd_ws)


## Create Cloud Compute Target 

In [4]:
from azureml.core.compute import ComputeTarget, BatchAiCompute
from azureml.core.compute_target import ComputeTargetException

# choose a name for your cluster
batchai_cluster_name = "traincluster"

try:
    # look for the existing cluster by name
    compute_target = ComputeTarget(workspace=ws, name=batchai_cluster_name)
    if type(compute_target) is BatchAiCompute:
        print('found compute target {}, just use it.'.format(batchai_cluster_name))
    else:
        print('{} exists but it is not a Batch AI cluster. Please choose a different name.'.format(batchai_cluster_name))
except ComputeTargetException:
    print('creating a new compute target...')
    compute_config = BatchAiCompute.provisioning_configuration(vm_size="STANDARD_NC6", # GPU VM or swap for standard dev machine using STANDARD_D2_V2 
                                                                #vm_priority='lowpriority', # optional
                                                                autoscale_enabled=True,
                                                                cluster_min_nodes=1, 
                                                                cluster_max_nodes=5)

    # create the cluster
    compute_target = ComputeTarget.create(ws, batchai_cluster_name, compute_config)

    # can poll for a minimum number of nodes and for a specific timeout. 
    # if no min node count is provided it uses the scale settings for the cluster
    compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)

    # Use the 'status' property to get a detailed status for the current cluster. 
    print(compute_target.status.serialize())

found compute target traincluster, just use it.


In [5]:
import os
script_folder = './keras-fashion'
os.makedirs(script_folder, exist_ok=True)

## Upload and use data in an Azure File Store

In [ ]:
ds = ws.get_default_datastore()
print(ds.datastore_type, ds.account_name, ds.container_name)

ds.upload(src_dir='./data', target_path='fashiondata', overwrite=True, show_progress=True)

## Write train.py file and save model.h5

In [7]:
%%writefile $script_folder/train.py

import tensorflow as tf
import os
import time
os.environ["TF_CPP_MIN_LOG_LEVEL"]= "2"
print("tensorflow Version is: " + str(tf.__version__))

import numpy as np
os.environ['KERAS_BACKEND'] = 'tensorflow'
from keras import backend as K
print(os.environ['KERAS_BACKEND'])

#Fashion MNIST Dataset CNN model development: https://github.com/zalandoresearch/fashion-mnist
from keras.datasets import fashion_mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import utils, losses, optimizers
from sklearn.externals import joblib

import azureml
from azureml.core import Workspace, Run

# let user feed in 2 parameters, the location of the data files (from datastore), and the regularization rate of the logistic regression model
parser = argparse.ArgumentParser()
parser.add_argument('--data-folder', type=str, dest='data_folder', help='data folder mounting point')
args = parser.parse_args()

data_folder = os.path.join(args.data_folder, 'keras-fashion')
print('Data folder:', data_folder)

#variables
num_classes = 10
batch_size = 128
epochs = 24
img_rows,img_cols = 28,28

#data for train and testing
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

# Define the text labels
fashion_mnist_labels = ["Top",          # index 0
                        "Trouser",      # index 1
                        "Jumper",       # index 2 
                        "Dress",        # index 3 
                        "Coat",         # index 4
                        "Sandal",       # index 5
                        "Shirt",        # index 6 
                        "Trainer",      # index 7 
                        "Bag",          # index 8 
                        "Ankle boot"]   # index 9


print(x_train.shape, y_train.shape, x_test.shape, y_test.shape, sep = '\n')

#data pre-processing
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
y_train = utils.to_categorical(y_train, num_classes)
y_test = utils.to_categorical(y_test,  num_classes)

#formatting issues for depth of image (greyscale = 1) with different kernels (tensorflow, cntk, etc)
if K.image_data_format()== 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0],1,img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols,1)
    x_test = x_test.reshape(x_test.shape[0],img_rows, img_cols,1)
    input_shape = (img_rows, img_cols,1)
    
#Define the CNN model
model = Sequential()

model.add(Conv2D(64, kernel_size=(3,3), padding = 'same', activation = 'relu', input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(32, kernel_size=(3,3), padding = 'same', activation = 'relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(128, activation='relu'))

model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.summary()

# get hold of the current run
run = Run.get_submitted_run()

print('Train a deep learning model')
model.compile(loss=losses.categorical_crossentropy, optimizer=optimizers.Adam(), metrics=['accuracy'])
hist = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(x_test, y_test))

#evaluate the model on the test data
print('Predict the test set')
score = model.evaluate(x_test, y_test, verbose=0)
print('Test Loss: ', score[0])
print('Test Accuracy: ', score[1])

# calculate accuracy on the prediction
print('Accuracy is', score[1])

run.log('accuracy', np.float(score[1]))

os.makedirs('outputs', exist_ok=True)
# note file saved in the outputs folder is automatically uploaded into experiment record
model.save('outputs/model.h5')


Overwriting ./keras-fashion/train.py


## Run the model in the cloud for training

In [8]:
import shutil
shutil.copy('utils.py', script_folder)

'./keras-fashion\\utils.py'

In [9]:
from azureml.train.estimator import Estimator

script_params = {
    #'--data-folder': ds.as_mount()
}

est = Estimator(source_directory=script_folder,
                script_params=script_params,
                compute_target=compute_target,
                entry_script='train.py',
                conda_packages=['keras', 'scikit-learn'])

In [20]:
run = exp.submit(config=est)
run

Experiment,Id,Type,Status,Details Page,Docs Page
deeplearning_fashion,deeplearning_fashion_1539252352993,azureml.scriptrun,Running,Link to Azure Portal,Link to Documentation


In [16]:
from azureml.train.widgets import RunDetails
RunDetails(run)
RunDetails(run).show()

_UserRun()

In [21]:
run.wait_for_completion(show_output=True) # specify True for a verbose log

RunId: deeplearning_fashion_1539252352993

Streaming azureml-logs/80_driver_log.txt

tensorflow Version is: 1.10.0
Using TensorFlow backend.
tensorflow
Data folder: /mnt/batch/tasks/shared/LS_root/jobs/trainclustera1358531bd/azureml/deeplearning_fashion_1539252352993/mounts/workspacefilestore/keras-fashion

32768/29515 [=================================] - 0s 0us/step

26427392/26421880 [==============================] - 0s 0us/step

8192/5148 [===============================================] - 0s 0us/step

4423680/4422102 [==============================] - 0s 0us/step
(60000, 28, 28)
(60000,)
(10000, 28, 28)
(10000,)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 28, 28, 64)        640       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 64)        0         
____________________________________________

ExperimentExecutionException: The output streaming for the run interrupted.
But the run is still executing on the compute target. 
To cancel the run, execute: az ml run cancel -r deeplearning_fashion_1539252352993 -t traincluster -p C:\Users\amynic\Documents\CloudDeveloperAdvocate\Deeplearning\ai-world-summit\azureml-sdk-fashion\keras-fashion

## Evaluate the model output

In [14]:
print(run.get_metrics())

{'accuracy': 0.9211}


In [15]:
print(run.get_file_names())

['azureml-logs/60_control_log.txt', 'azureml-logs/80_driver_log.txt', 'outputs/model.h5', 'driver_log', 'azureml-logs/azureml.log', 'azureml-logs/55_batchai_execution.txt']


## Are you happy with the model??? Register it in Azure Machine Learning to manage

In [16]:
# register model 
model = run.register_model(model_name='keras_dl_fashion', model_path='outputs/')
print(model.name, model.id, model.version, sep = '\t')

keras_dl_fashion_test	keras_dl_fashion_test:2	2


## Tidy up Compute Resource when not in use

In [ ]:
# optionally, delete the Azure Managed Compute cluster
compute_target.delete()